In [1]:
#Loading necessary packages
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import matplotlib.pyplot as plt


In [2]:
#Load the dataset
data = pd.read_csv("MER_T12_06.csv")
data.head()

,MSN,YYYYMM,Value,Column_Order,Description,Unit
0,CLEIEUS,197301,72.076,1,Coal Electric Power Sector CO2 Emissions,Million Metric Tons of Carbon Dioxide
1,CLEIEUS,197302,64.442,1,Coal Electric Power Sector CO2 Emissions,Million Metric Tons of Carbon Dioxide
2,CLEIEUS,197303,64.084,1,Coal Electric Power Sector CO2 Emissions,Million Metric Tons of Carbon Dioxide
3,CLEIEUS,197304,60.842,1,Coal Electric Power Sector CO2 Emissions,Million Metric Tons of Carbon Dioxide
4,CLEIEUS,197305,61.798,1,Coal Electric Power Sector CO2 Emissions,Million Metric Tons of Carbon Dioxide


In [3]:
#Preprocessing the dataset
data = data[['YYYYMM', 'Value']] # keeping only the YYYYMM and Value columns
data['YYYYMM'] = data['YYYYMM'].astype(str).str[:4].astype(int) # keeping only the year
data = data.rename(columns={'YYYYMM': 'Year', 'Value': 'Value'}) # renaming the columns to increase readability

In [4]:
#Replacing missing values with NaN
data = data.replace('Not Available', np.nan)

In [5]:
#Replacing NaN values with the mean value
mean_value = data['Value'].astype(float).mean()
data['Value'] = data['Value'].astype(float).fillna(mean_value)

In [6]:
#Splitting the data into training and test sets based on Year column
train_data = data[data['Year'] < 2015]
test_data = data[data['Year'] >= 2015]

In [7]:
#Scaling the data
scaler = MinMaxScaler()
train_data = scaler.fit_transform(train_data)
test_data = scaler.transform(test_data)


In [8]:
#Splitting the data into features and targets
x_train = train_data[:, :-1]
y_train = train_data[:, -1]
x_test = test_data[:, :-1]
y_test = test_data[:, -1]

In [9]:
#Reshaping the data to be suitable for LSTM
x_train = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
x_test = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))

In [10]:
#Building the LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(1))

In [11]:
#Compiling the model
model.compile(optimizer='adam', loss='mean_squared_error')


In [12]:
#Fitting the model
history = model.fit(x_train, y_train, batch_size=1, epochs=100)


Epoch 1/100
4914/4914 [==============================] - 32s 5ms/step - loss: 0.0091
Epoch 2/100
4914/4914 [==============================] - 27s 6ms/step - loss: 0.0091
Epoch 3/100
4914/4914 [==============================] - 27s 6ms/step - loss: 0.0091
Epoch 4/100
4914/4914 [==============================] - 27s 6ms/step - loss: 0.0091
Epoch 5/100
4914/4914 [==============================] - 27s 6ms/step - loss: 0.0091
Epoch 6/100
4914/4914 [==============================] - 27s 5ms/step - loss: 0.0091
Epoch 7/100
4914/4914 [==============================] - 27s 5ms/step - loss: 0.0091
Epoch 8/100
4914/4914 [==============================] - 27s 6ms/step - loss: 0.0091
Epoch 9/100
4914/4914 [==============================] - 27s 5ms/step - loss: 0.0091
Epoch 10/100
4914/4914 [==============================] - 24s 5ms/step - loss: 0.0091
Epoch 11/100
4914/4914 [==============================] - 24s 5ms/step - loss: 0.0091
Epoch 12/100
4914/4914 [==============================] - 27s 6

In [13]:
#Predictting the test set
y_pred = model.predict(x_test)

6/6 [==============================] - 1s 3ms/step


In [14]:
                   
# Inverse transforming the data to get the original scale
y_test = np.hstack((y_test.reshape(-1, 1), np.zeros((y_test.shape[0], 1))))
y_pred = np.hstack((y_pred.reshape(-1, 1), np.zeros((y_pred.shape[0], 1))))

In [15]:
y_test = scaler.inverse_transform(y_test)
y_pred = scaler.inverse_transform(y_pred)

In [16]:
#MSE is calculated by comparing the predicted values (y_pred) with the original test values (y_test).
mse = np.mean((y_pred - y_test)**2)
print('Test MSE: %.2f' % mse)

Test MSE: 5.02


In [17]:
# Calculating the mean squared error
mse = np.mean((y_pred - y_test)**2)

In [18]:
# Calculating the root mean squared error
rmse = np.sqrt(mse)

print('Test RMSE: %.2f' % rmse)

Test RMSE: 2.24
